In [1]:
import sys
sys.path.append('./deep')

In [2]:
import numpy as np
W = np.arange(21).reshape(7,3)

W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [ ]:
W[2]

array([6, 7, 8])

In [4]:
W[3]

array([ 9, 10, 11])

In [ ]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.embed_W = W
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        min_val = np.finfo(dW.dtype).min
        max_val = np.finfo(dW.dtype).max
        np.clip(dW, min_val, max_val, out=dW)
        return None

1
1
1
{'leo': 1}
dict_items([('leo', 1)])
dict_keys(['leo'])
<class 'dict_keys'>
leo
